In [1]:
# All required python standard libraries
import os
import time

In [2]:
# All torch related imports 
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from torch import nn, optim
import torchvision

In [3]:
# using cv2 to read an image
import cv2

In [4]:
# All sci-kit related imports 
import pandas as pd
import numpy as np

In [5]:
from tqdm.notebook import tqdm as tq

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
IMAGE_DIMS = 224

In [8]:
def accuracy_finder(predictions , labels):
    values, max_indices = torch.max(predictions, dim=1)
    accuracy = ( max_indices == labels ).sum()
    return accuracy/max_indices.size()[0]

In [9]:
def csv_preprocessor(base_dir:str, directory:str):
    return os.path.join(base_dir,directory).replace("\\","/")

In [10]:
def return_all_image_list_from_processed_csv(csv_file):
    ### This returns the entire list full of images to be loaded into cpu
    ###
    ###
    ALL_IMAGES = []
    start = time.time()
    for i, items in tq(enumerate(csv_file)):
        image = cv2.imread(items, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(image,(IMAGE_DIMS,IMAGE_DIMS))
        ALL_IMAGES.append(resized)
    
    print("Tt took us approximately {} seconds".format(time.time()-start))  
    return ALL_IMAGES

In [11]:
def get_one_hot_encoded_labels(input_data_frame):
    input_data_frame.labels = input_data_frame.labels.map(lambda x: x-1)
    # return pd.get_dummies(input_data_frame.labels, prefix='labels').to_numpy()
    return input_data_frame.labels

In [12]:
# pytorch device configurations 
BATCH_SIZE = 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
DEVICE

device(type='cuda')

In [14]:
DATASET = pd.read_pickle('./sync_pickles/training_set.pkl')

In [16]:
X_TRAIN_CSV, X_VAL_CSV,Y_TRAIN, Y_VAL = train_test_split(DATASET.images, DATASET.labels, test_size=0.20, stratify=DATASET.labels)

In [17]:
print(len(X_TRAIN_CSV),len(X_VAL_CSV),len(Y_TRAIN), len(Y_VAL))

27551 6888 27551 6888


In [18]:
DATA_NORMALIZER = transforms.Compose([transforms.ToTensor(),transforms.Resize((IMAGE_DIMS,IMAGE_DIMS)),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

In [19]:
class LeanIsolatedCharacterDataset(Dataset):
    def __init__(self, images, labels,  transforms=None):
          
        ### labels
        self.labels = labels.to_numpy()
        
        self.ALL_IMAGES = images.to_numpy()
        
        ### transformations to apply on images
        self.transforms = transforms
        
    def __getitem__(self, index):
        # convert labels to tensor 
        label = torch.tensor(self.labels[index])
        # load single image from list of all preloaded images
        image = self.ALL_IMAGES[index]
        if self.transforms:
            ## apply transforms 
            image = self.transforms(image)    
            image = image.float()
        label = label.long()
        return image, label 
    
    def __len__(self):
        return len(self.ALL_IMAGES)

In [20]:
TRAINING_DATASET = LeanIsolatedCharacterDataset(images=X_TRAIN_CSV, labels=Y_TRAIN, transforms=DATA_NORMALIZER)

In [21]:
len(TRAINING_DATASET)

27551

In [22]:
VALIDATION_DATASET = LeanIsolatedCharacterDataset(images=X_VAL_CSV, labels=Y_VAL, transforms=DATA_NORMALIZER)

In [23]:
len(VALIDATION_DATASET)

6888

In [24]:
TRAINING_LOADER = DataLoader(dataset=TRAINING_DATASET,batch_size=BATCH_SIZE,shuffle=True)
VALIDATION_LOADER = DataLoader(dataset=VALIDATION_DATASET,batch_size=BATCH_SIZE,shuffle=True)

In [25]:
class block(nn.Module):
    def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride=stride, padding=1,)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(intermediate_channels, intermediate_channels * self.expansion, kernel_size=1, stride=1, padding=0,)
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride
    
    #Identity block
    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        #x = self.relu(x) #custom 

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [26]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels change
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, intermediate_channels * 4, kernel_size=1, stride=stride,),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride))

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [27]:
def ResNet50(img_channel=3, num_classes=171):
    return ResNet(block, [2, 3, 5, 2], img_channel, num_classes)
net = ResNet50()

In [28]:
optimizer =  optim.Adam(net.parameters(), lr=0.001) # learning rate 
# defining the loss function
criterion =  nn.CrossEntropyLoss() # reduction='none'
net = net.to(DEVICE)
criterion = criterion.to(DEVICE)

In [29]:
from tqdm import tqdm 

In [30]:
def validate(ValidationLoader):
    total_val_accuracy = 0
    total_val_loss = 0 
    
    for i, data in enumerate(ValidationLoader, 0):
       # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        total_val_loss += loss
        val_accuracy = accuracy_finder(predictions=outputs, labels=labels)
        total_val_accuracy = total_val_accuracy + val_accuracy
        
    return (total_val_loss/len(ValidationLoader), total_val_accuracy/len(ValidationLoader))

In [35]:
def training(epochs:int = 0, TRAINING_LOADER = TRAINING_LOADER , VALIDATION_LOADER = VALIDATION_LOADER, OPTIM_MODEL_NAME="" ):
    all_training_losses = []
    all_training_accuracy = []
    
    all_validation_losses = []
    all_validation_accuracy = []
    
    minimum_validation_loss = torch.tensor(999999999).to(DEVICE)
    minimum_validation_epoch_number = -1
    
    
    for epoch in tq(range(epochs)):
        total_epoch_loss = 0
        total_accuracy_epoch = 0
        for i, data in tqdm(enumerate(TRAINING_LOADER, 0)): 
            
            image = 0
            label = 0
            
            image,label = data
            
            optimizer.zero_grad(set_to_none=True)
            
            label = label.to(DEVICE)
            image = image.to(DEVICE)
            output = net(image)
            
            
            
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                total_epoch_loss += loss
                batches_training_accuracy = accuracy_finder(predictions=output, labels=label)
                total_accuracy_epoch = total_accuracy_epoch  + batches_training_accuracy   
            
            if i % 1000 == 0 and i != 0: 
                print("Batch : {}/{}".format(i, len(TRAINING_LOADER)))
           
        # total epoch loss 
        total_epoch_loss = total_epoch_loss / len(TRAINING_LOADER)
        # total epoch accuracy 
        total_accuracy_epoch = total_accuracy_epoch /len(TRAINING_LOADER)
        
        optimizer.zero_grad(set_to_none=True)
        
        inputs = 0
        image = 0
        output = 0
        
        with torch.no_grad():
             validation_loss, validation_accuracy = validate(ValidationLoader= VALIDATION_LOADER )
        
        all_validation_losses.append(validation_loss)
        all_validation_accuracy.append(validation_accuracy)
        
        if minimum_validation_loss > validation_loss:
            minimum_validation_loss = validation_loss
            minimum_validation_epoch_number = epoch
            print("Saved model at epoch {}".format(epoch+1))
            PATH = OPTIM_MODEL_NAME
            # saving model
            torch.save(net.state_dict(), PATH)
            
        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.8f}, acc = {:.8f}, val_loss = {:.8f} , val_acc = {:.8f}, ".format(epoch + 1, epochs, total_epoch_loss, total_accuracy_epoch, validation_loss, validation_accuracy ))
        all_training_losses.append(total_epoch_loss)
        all_training_accuracy.append(total_accuracy_epoch)
        
    print("Training completed")
    return all_training_accuracy, all_training_losses,all_validation_accuracy,all_validation_losses,minimum_validation_epoch_number, minimum_validation_loss

In [36]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)
MODEL_NAME = "pretrained_resnet_50"
# dd/mm/YY H:M:S
ExcecutionTime = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", ExcecutionTime)

now = 2021-04-17 01:56:46.281846
date and time = 17/04/2021 01:56:46


In [37]:
PATH = "./models"
OPTIM_MODEL_NAME_PATH = "{}/{}_{}_minimum_validation_loss.pth".format(PATH,MODEL_NAME,ExcecutionTime)

In [38]:
t_acc, t_loss, v_acc, v_loss, min_val_loss, min_val_loss_epoch = training(100, TRAINING_LOADER = TRAINING_LOADER , VALIDATION_LOADER = VALIDATION_LOADER, OPTIM_MODEL_NAME=OPTIM_MODEL_NAME_PATH)

1001it [07:35,  2.32it/s]

Batch : 1000/3444


2001it [14:39,  2.36it/s]

Batch : 2000/3444


3001it [21:42,  2.36it/s]

Batch : 3000/3444


3444it [24:50,  2.31it/s]


Saved model at epoch 1



OSError: [Errno 22] Invalid argument: './models/pretrained_resnet_50_17/04/2021 01:56:46_minimum_validation_loss.pth'

In [ ]:
# saving model
torch.save(net.state_dict(), MODEL_NAME_PATH)
print("model was successfully saved")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def result_plot(t_value,v_value,title,min_point, ylabel):
    result = []
    result_v = []
    for item, item_v in zip(t_value, v_value):
        result.append(item.cpu())
        result_v.append(item_v.cpu())
    plt.plot(result,'b')
    plt.plot(result_v,'r')
    if min_point is not None:
        x,y = min_point
        y = y.cpu()
        plt.plot(x,y,'xr',linewidth=2, markersize=12) 
    plt.title(title)
    plt.legend(["training","validation"])
    plt.xlabel("epochs")
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
result_plot(t_value=t_acc,v_value=v_acc,title="Accuracy",min_point=None, ylabel="accuracy")

In [ ]:
result_plot(t_value=t_loss,v_value=v_loss, title="Loss" ,min_point=(min_val_loss,min_val_loss_epoch), ylabel="loss")